# Use API to get data

Use public API to get data about covid19 confirmed cases by country

In [71]:
import requests
from datetime import datetime, timedelta

start = '2021-1-1'
end = str(datetime.today())

country = 'australia'

endpoint = f'https://api.covid19api.com/total/country/{country}/status/confirmed'
params = {'from': start, 'to': end}
res = requests.get(endpoint, params=params)

In [73]:
from pandas import DataFrame
df = DataFrame(res.json())

In [77]:
df['Cases'].tolist()

[56920,
 56968,
 57008,
 57034,
 57072,
 57092,
 57142,
 57164,
 57190,
 57228,
 57268,
 57300,
 57320,
 57338,
 57378,
 57416,
 57442,
 57462,
 57480,
 57498,
 57510,
 57520,
 57532,
 57554,
 57560,
 57572,
 57588,
 57598,
 57612,
 57622,
 57636,
 57646,
 57658,
 57676,
 57684,
 57696,
 57702,
 57714,
 57720,
 57742,
 57758,
 57774,
 57784,
 57796,
 57800,
 57810,
 57822,
 57824,
 57836,
 57840,
 57852,
 57860,
 57874,
 57878,
 57894,
 57914,
 57930,
 57940,
 57956,
 57972,
 57992,
 58014,
 58042,
 58058,
 58068,
 58092,
 58122,
 58148,
 58180,
 58204,
 29112,
 29117,
 29130,
 29137,
 29154,
 29166,
 29183,
 29192,
 29196,
 29206,
 29211,
 29221,
 29230,
 29239,
 29252,
 29259,
 29276]

In [3]:
# we are going to make a request for every country on our list

# again import the function we have
import sys
sys.path.append("..")

from vaccination_project.vaccination_source import get_data_source_from_lws, content_to_df

In [4]:
countries = get_data_source_from_lws(0)
country_df = content_to_df(countries)

In [15]:
# this is going to take a while...
output = []
for country in country_df.country:
    print(country)
    endpoint = f'https://api.covid19api.com/total/country/{country}/status/confirmed'
    params = {'from': start, 'to': end}
    res = requests.get(endpoint, params=params)
    data = res.json()  # get back a list of every day record
    for each_record in data:
        try:
            cases = each_record['Cases']
            date = each_record['Date']
            output.append({"country": country, "cases": cases, "date": date})
        except Exception as err:
            print(f"Error for {country}:")
            print(err)
        

Afghanistan
Albania
Algeria
Andorra
Angola
Anguilla
Antigua and Barbuda
Argentina
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Bermuda
Bolivia
Brazil
Bulgaria
Error for Bulgaria:
string indices must be integers
Error for Bulgaria:
string indices must be integers
Cambodia
Error for Cambodia:
string indices must be integers
Error for Cambodia:
string indices must be integers
Canada
Cayman Islands
Chile
China
Colombia
Costa Rica
Cote d'Ivoire
Error for Cote d'Ivoire:
string indices must be integers
Croatia
Cyprus
Czechia
Denmark
Error for Denmark:
string indices must be integers
Error for Denmark:
string indices must be integers
Dominica
Dominican Republic
Ecuador
Egypt
El Salvador
England
Error for England:
string indices must be integers
Equatorial Guinea
Estonia
Faeroe Islands
Error for Faeroe Islands:
string indices must be integers
Falkland Islands
Error for Falkland Islands:
string indices must be integers
Finland
France
Germany
Ghana
Gibra

In [19]:
import pandas as pd
df = pd.DataFrame(output)

In [28]:
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')

In [30]:
df.head()

,country,cases,date
0,Afghanistan,51526,2021-01-01
1,Afghanistan,51526,2021-01-02
2,Afghanistan,51526,2021-01-03
3,Afghanistan,53011,2021-01-04
4,Afghanistan,53105,2021-01-05


In [31]:
df['id'] = df['country'] + df['date']

In [32]:
df

,country,cases,date,id
0,Afghanistan,51526,2021-01-01,Afghanistan2021-01-01
1,Afghanistan,51526,2021-01-02,Afghanistan2021-01-02
2,Afghanistan,51526,2021-01-03,Afghanistan2021-01-03
3,Afghanistan,53011,2021-01-04,Afghanistan2021-01-04
4,Afghanistan,53105,2021-01-05,Afghanistan2021-01-05
...,...,...,...,...
10229,Zimbabwe,36717,2021-03-23,Zimbabwe2021-03-23
10230,Zimbabwe,36749,2021-03-24,Zimbabwe2021-03-24
10231,Zimbabwe,36778,2021-03-25,Zimbabwe2021-03-25
10232,Zimbabwe,36805,2021-03-26,Zimbabwe2021-03-26


In [42]:
from vaccination_project.database import get_data_from_db, upload_df_to_db

In [40]:
upload_df_to_db(df, "vax.db", "confirmed_cases")

Updated confirmed_cases


In [43]:
qry = "select * from confirmed_cases"
get_data_from_db("vax.db", qry)

,country,cases,date,id
0,Afghanistan,51526,2021-01-01,Afghanistan2021-01-01
1,Afghanistan,51526,2021-01-02,Afghanistan2021-01-02
2,Afghanistan,51526,2021-01-03,Afghanistan2021-01-03
3,Afghanistan,53011,2021-01-04,Afghanistan2021-01-04
4,Afghanistan,53105,2021-01-05,Afghanistan2021-01-05
...,...,...,...,...
10229,Zimbabwe,36717,2021-03-23,Zimbabwe2021-03-23
10230,Zimbabwe,36749,2021-03-24,Zimbabwe2021-03-24
10231,Zimbabwe,36778,2021-03-25,Zimbabwe2021-03-25
10232,Zimbabwe,36805,2021-03-26,Zimbabwe2021-03-26


In [66]:
# Ok we get what we want!

master_qry = """
select * from vaccination_data as vd 
join countries as c 
on vd.country = c.country
left join confirmed_cases as cc 
on vd.country || vd.date = cc.id

"""
get_data_from_db('vax.db', master_qry)

,country,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,country,iso_code,vaccines,source_name,source_website,country,cases,date,id
0,Afghanistan,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,Afghanistan,55617.0,2021-02-22,Afghanistan2021-02-22
1,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,Afghanistan,55646.0,2021-02-23,Afghanistan2021-02-23
2,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,Afghanistan,55664.0,2021-02-24,Afghanistan2021-02-24
3,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,Afghanistan,55680.0,2021-02-25,Afghanistan2021-02-25
4,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,Afghanistan,55696.0,2021-02-26,Afghanistan2021-02-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8446,Zimbabwe,2021-03-23,44135.0,44135.0,NaN,841.0,655.0,0.30,0.30,NaN,44.0,Zimbabwe,ZWE,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1375899540...,Zimbabwe,36717.0,2021-03-23,Zimbabwe2021-03-23
8447,Zimbabwe,2021-03-24,49404.0,49404.0,NaN,5269.0,1400.0,0.33,0.33,NaN,94.0,Zimbabwe,ZWE,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1375899540...,Zimbabwe,36749.0,2021-03-24,Zimbabwe2021-03-24
8448,Zimbabwe,2021-03-25,54892.0,54892.0,NaN,5488.0,2127.0,0.37,0.37,NaN,143.0,Zimbabwe,ZWE,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1375899540...,Zimbabwe,36778.0,2021-03-25,Zimbabwe2021-03-25
8449,Zimbabwe,2021-03-26,61093.0,61093.0,NaN,6201.0,2772.0,0.41,0.41,NaN,187.0,Zimbabwe,ZWE,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1375899540...,Zimbabwe,36805.0,2021-03-26,Zimbabwe2021-03-26


In [67]:
master_df = get_data_from_db("vax.db", master_qry)

In [68]:
master_df.head()

,country,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,country,iso_code,vaccines,source_name,source_website,country,cases,date,id
0,Afghanistan,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,Afghanistan,55617.0,2021-02-22,Afghanistan2021-02-22
1,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,Afghanistan,55646.0,2021-02-23,Afghanistan2021-02-23
2,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,Afghanistan,55664.0,2021-02-24,Afghanistan2021-02-24
3,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,Afghanistan,55680.0,2021-02-25,Afghanistan2021-02-25
4,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Afghanistan,AFG,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,Afghanistan,55696.0,2021-02-26,Afghanistan2021-02-26


# Wrap up in functions

In [61]:
import requests
from datetime import datetime
import pandas as pd

def get_confirmed_cases_api(countries):
    start = '2021-1-1'
    end = str(datetime.today())
    output = []
    for country in countries:
        print(country)
        endpoint = f'https://api.covid19api.com/total/country/{country}/status/confirmed'
        params = {'from': start, 'to': end}
        res = requests.get(endpoint, params=params)
        data = res.json()  # return a list of every day record
        for each_record in data:
            try:
                cases = each_record['Cases']
                date = each_record['Date']
                output.append({"country": country, "cases": cases, "date": date})
            except Exception as err:
                print(f"Error for {country}:")
                print(err)
    # convert to dataframe
    df = pd.DataFrame(output)
    
    # modify date format for 'date' column
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
    # creage a column called id
    df['id'] = df['country'] + df['date']
    
    return df
        